<a href="https://colab.research.google.com/github/redadmiral/python-for-journalists/blob/main/lessons/PandasDataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning

Nachdem wir in der Explorativen Analyse gesehen haben, wie unser Datensatz aufgebaut ist und welche Probleme er hat, machen wir uns nun daran, den Datensatz zu bereinigen um sicher damit arbeiten zu können.

Den Datensatz speichern wir dann anschließend ab, um die Analyse auf dem bereinigten Datensatz durchführen zu können.

# Datensatz reduzieren

Zuerst lesen wir den Datensatz ein und wiederholen wir den Schritt aus der explorativen Analyse und reduzieren ihn auf die von uns benötigten Spalten:

In [1]:
import pandas as pd

dogs = pd.read_csv("KUL100OD1001.csv")

dogs = dogs[['StichtagDatJahr', 'HalterId', 'AlterV10Cd', 'SexLang', 'KreisCd', 'KreisLang', 'QuarCd', 'QuarLang', 'Rasse1Text',
       'GebDatHundJahr', 'AlterVHundCd', 'SexHundLang', 'HundefarbeText']]

dogs

,StichtagDatJahr,HalterId,AlterV10Cd,SexLang,KreisCd,KreisLang,QuarCd,QuarLang,Rasse1Text,GebDatHundJahr,AlterVHundCd,SexHundLang,HundefarbeText
0,2015,126,60,männlich,9,Kreis 9,92,Altstetten,Welsh Terrier,2011,3,weiblich,schwarz/braun
1,2015,574,60,weiblich,2,Kreis 2,23,Leimbach,Cairn Terrier,2002,12,weiblich,brindle
2,2015,695,40,männlich,6,Kreis 6,63,Oberstrass,Labrador Retriever,2012,2,weiblich,braun
3,2015,893,60,weiblich,7,Kreis 7,71,Fluntern,Mittelschnauzer,2010,4,weiblich,schwarz
4,2015,1177,50,männlich,10,Kreis 10,102,Wipkingen,Shih Tzu,2011,3,männlich,schwarz/weiss
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61467,2022,160175,20,weiblich,8,Kreis 8,82,Mühlebach,Chihuahua,2021,0,weiblich,hellbraun
61468,2022,160176,20,weiblich,8,Kreis 8,83,Weinegg,Zwergpudel,2012,9,weiblich,apricot
61469,2022,160178,50,männlich,11,Kreis 11,111,Affoltern,Shetland Sheepdog,2022,0,weiblich,tricolor
61470,2022,160182,30,weiblich,7,Kreis 7,72,Hottingen,Miniature American Shepard,2022,0,männlich,braun


## Ersetzen von falschen Werten

In der explorativen Analyse haben wir herausgefunden, dass die Rasse des Hundes manchmal als Unbekannt angegeben ist und das Alter des Hundes in acht Fällen falsch ist.

Für kategorielle Datenreihen wie die Rasse können wir, müssen wir den Eintrag nicht ersetzen: Wir würden unsere Auswertung nicht verfälschen wenn wir den Eintrag "Unbekannt" in `Rasse1Text` beibehalten.

Tatsächlich würden wir ein wenig an Informationen verlieren, weil wir so wissen, dass 7856 Hundehalter\*innen die Rasse ihres Hundes nicht kennen. Deshalb belassen wir den Eintrag `Unbekannt`.

Anders sieht es beim Alter der Hunde aus. Wenn wir uns etwa für das durchschnittliche Alter des Hundes interessieren, dann verfälschen die acht Hunde mit dem Alter 999 unser Ergebnis.

Mit den acht Einträgen erhalten wir als durchschnittliches Alter über alle Jahre hinweg (was kein sinnvolles Maß ist):

In [2]:
dogs["AlterVHundCd"].mean()

6.080052706923477

Um die Werte die wir als fehlerhafte Eingaben identifiziert haben gegen `pd.NA` auszutauschen, das etwa von der `.mean()`-Methode einfach ignoriert wird können wir die `.replace()`-Methode verwenden.

Sie ersetzt in einem Dataframe oder einer Series einen Wert durch einen anderen. 


In [3]:
dogs["AlterVHundCd"] = dogs["AlterVHundCd"].replace(999, pd.NA)

Wenn wir nun wieder alle Werte in der Spalte ansehen, sehen wir dass die falschen Einträge verschwunden sind:

In [4]:
dogs["AlterVHundCd"].value_counts()

1     5463
0     5312
2     5151
3     5070
4     4905
5     4743
6     4540
7     4387
8     4139
9     3871
10    3566
11    3156
12    2599
13    1991
14    1313
15     731
16     334
17     122
18      41
19      12
20       9
21       6
22       2
23       1
Name: AlterVHundCd, dtype: int64

Das gleiche machen wir noch einmal mit dem Eintrag für das Geburtsjahr:

In [5]:
dogs["GebDatHundJahr"] = dogs["GebDatHundJahr"].replace(9999, pd.NA)

Und auch das durchschnittliche Alter ist nun ein anderes:

In [6]:
dogs["AlterVHundCd"].mean()

5.950816738253287

Manchmal macht es Sinn, fehlerhafte Einträge zu löschen oder zu versuchen, die fehlenden Informationen zu ersetzen.

Es könnte etwa sein, dass das Alter des Hundes nur einmal vergessen wurde und in einem anderen Jahr eingetragen wurde.

Dafür werfen wir einen Blick auf alle fehlerhaften Einträge, die wir nun einfach mit der `.isna()`-Methode finden können:

In [7]:
dogs[dogs["AlterVHundCd"].isna()]

,StichtagDatJahr,HalterId,AlterV10Cd,SexLang,KreisCd,KreisLang,QuarCd,QuarLang,Rasse1Text,GebDatHundJahr,AlterVHundCd,SexHundLang,HundefarbeText
2632,2015,91205,40,männlich,6,Kreis 6,61,Unterstrass,Zwergschnauzer,<NA>,<NA>,weiblich,schwarz/silber
4525,2015,111954,50,weiblich,3,Kreis 3,34,Sihlfeld,Parson Russell Terrier,<NA>,<NA>,männlich,tricolor
5584,2015,121585,40,männlich,7,Kreis 7,74,Witikon,Malteser,<NA>,<NA>,männlich,weiss
9429,2016,91205,40,männlich,6,Kreis 6,61,Unterstrass,Zwergschnauzer,<NA>,<NA>,weiblich,schwarz/silber
11247,2016,111954,50,weiblich,3,Kreis 3,34,Sihlfeld,Parson Russell Terrier,<NA>,<NA>,männlich,tricolor
16151,2017,91205,40,männlich,6,Kreis 6,61,Unterstrass,Zwergschnauzer,<NA>,<NA>,weiblich,schwarz/silber
23071,2018,91205,40,männlich,6,Kreis 6,61,Unterstrass,Zwergschnauzer,<NA>,<NA>,weiblich,schwarz/silber
35341,2019,143717,30,weiblich,12,Kreis 12,122,Schwamendingen-Mitte,Bolonka Zwetna,<NA>,<NA>,männlich,rotbraun


Wir sehen, dass es sich nur um vier verschiedene Halter handelt. Wir können sie also einfach alle durchprobieren um zu sehen, ob wir recht haben:

In [11]:
dogs[dogs["HalterId"] == 143717]

,StichtagDatJahr,HalterId,AlterV10Cd,SexLang,KreisCd,KreisLang,QuarCd,QuarLang,Rasse1Text,GebDatHundJahr,AlterVHundCd,SexHundLang,HundefarbeText
35341,2019,143717,30,weiblich,12,Kreis 12,122,Schwamendingen-Mitte,Bolonka Zwetna,<NA>,<NA>,männlich,rotbraun


Leider können wir bei keinem der Hunde das wahre Alter herausfinden. 

Eine andere Option ist es immer noch die fehlerhaften Einträge ganz zu löschen, aber in diesem Fall schränken sie unsere Analyse nicht ein. Deshalb können wir die Einträge beibehalten.

Zum Ende können wir die Funktion `.describe()` auf dem Datensatz ausführen. Diese zeigt uns noch einen kleinen Überblick über den Datensatz. 

In [12]:
dogs.describe()

,StichtagDatJahr,HalterId,AlterV10Cd,KreisCd,QuarCd
count,61472.000000,61472.000000,61472.000000,61472.000000,61472.000000
mean,2018.710974,113860.603478,48.316551,7.400442,76.575189
std,2.311283,25806.372691,58.500427,3.312585,33.549868
min,2015.000000,126.000000,10.000000,1.000000,10.000000
25%,2017.000000,89705.000000,30.000000,4.000000,44.000000
50%,2019.000000,116324.000000,40.000000,8.000000,81.000000
75%,2021.000000,133157.000000,60.000000,10.000000,102.000000
max,2022.000000,160183.000000,999.000000,99.000000,999.000000


Die Tabelle ist sehr umfangreich, aber die Werte sind schnell erklärt:

- count ist Zahl der Zeilen
- mean ist der durchschnittliche Wert in der Spalte
- std ist die Standardabweichung. Je kleiner diese Zahl ist, desto weniger verteilt sind sie.
- min/max: Der Minimal- bzw. Maximalwert der Spalte
- 25%/50%/75%: Oberer Wert des Quartils. Wenn man den Datensatz sortiert und in vier Teile teilt, würde diese Zahl als jeweils letzte in jedem der Viertel stehen.

Hier sehen wir, dass auch die Spalten `AlterV10Cd`, `KreisCd` und	`QuarCd` sehr hohe Maximalwerte haben. Ein Blick auf die [Quartierliste Zürichs](https://de.wikipedia.org/wiki/Stadtteile_der_Stadt_Z%C3%BCrich) zeigt, dass es 12 Kreise und 22 Stadtquartieren bzw. 34 Statistische Quartiere gibt. 

Wir können also davon ausgehen, dass die 99 bzw. 999 hier auch wieder bedeuten, dass es keine Einträge gibt und können sie durch `pd.NA` ersetzen.

Um weniger tippen zu müssen, können wir alle drei Spalten gleichzeitig ändern. Alternativ zu `.replace()` benutzen wir hier die `mask`-Methode. 

Dafür erstellen wir zuerst eine Maske, die wie gehabt ein DataFrame aus True/False-Werten ist.

In [15]:
mask = dogs[["AlterV10Cd", "KreisCd", "QuarCd"]].isin([99, 999])

Diese Maske wenden wir dann mit der `mask()`-Methode auf den DataFrame an:

In [16]:
dogs[["AlterV10Cd", "KreisCd", "QuarCd"]] = dogs[["AlterV10Cd", "KreisCd", "QuarCd"]].mask(mask, pd.NA)

Die `mask()`-Funktion ist mächtiger als das reine `replace()`, weil wir statt eines Werts, wie hier `pd.NA` auch einen zweiten DataFrame mit den gleichen Abmessungen übergeben können. Mask tauscht dann an allen Stellen an denen die Maske `True` ist die Werte aus dem ersten DataFrame mit den Werten aus dem zweiten DataFrame aus.

Wenn wir jetzt nochmals die `describe()`-Methode aufrufen, sehen wir dass die falschen 99 bzw. 999 Werte verschwunden sind.

In [17]:
dogs.describe()

,StichtagDatJahr,HalterId,AlterV10Cd,KreisCd,QuarCd
count,61472.000000,61472.000000,61257.000000,61470.000000,61470.000000
mean,2018.710974,113860.603478,44.979839,7.397462,76.545177
std,2.311283,25806.372691,15.842023,3.271172,33.135252
min,2015.000000,126.000000,10.000000,1.000000,10.000000
25%,2017.000000,89705.000000,30.000000,4.000000,44.000000
50%,2019.000000,116324.000000,40.000000,8.000000,81.000000
75%,2021.000000,133157.000000,60.000000,10.000000,102.000000
max,2022.000000,160183.000000,90.000000,12.000000,123.000000


Hier fällt uns etwas weiteres auf: 

Es sind nicht alle Spalten aus `dogs` in der Auswertung von `.describe()`. Das liegt daran, dass `describe()` Zahlen priorisiert. 

Der Blick auf die Datenypem mit `dtypes` zeigt, dass die zuvor mit `replace` und `mask` bearbeiteten Spalten aber einen anderen Typ haben. 

In [18]:
dogs.dtypes

StichtagDatJahr      int64
HalterId             int64
AlterV10Cd         float64
SexLang             object
KreisCd            float64
KreisLang           object
QuarCd             float64
QuarLang            object
Rasse1Text          object
GebDatHundJahr      object
AlterVHundCd        object
SexHundLang         object
HundefarbeText      object
dtype: object

Deshalb ändern wir den Datentyp der Spalten mit der `.astype()`-Methode.

In [20]:
dogs[["AlterVHundCd", "GebDatHundJahr", "QuarCd", "KreisCd", "AlterV10Cd"]] = dogs[["AlterVHundCd", "GebDatHundJahr", "QuarCd", "KreisCd", "AlterV10Cd"]].astype('Int64')

Und jetzt tauchen alle numerischen Spalten wieder in der Ausgabe von `describe()` auf:

In [21]:
dogs.describe()

,StichtagDatJahr,HalterId,AlterV10Cd,KreisCd,QuarCd,GebDatHundJahr,AlterVHundCd
count,61472.000000,61472.000000,61257.000000,61470.000000,61470.000000,61464.000000,61464.000000
mean,2018.710974,113860.603478,44.979839,7.397462,76.545177,2011.763829,5.950817
std,2.311283,25806.372691,15.842023,3.271172,33.135252,4.812745,4.164937
min,2015.000000,126.000000,10.000000,1.000000,10.000000,1994.000000,0.000000
25%,2017.000000,89705.000000,30.000000,4.000000,44.000000,2008.000000,2.000000
50%,2019.000000,116324.000000,40.000000,8.000000,81.000000,2012.000000,6.000000
75%,2021.000000,133157.000000,60.000000,10.000000,102.000000,2015.000000,9.000000
max,2022.000000,160183.000000,90.000000,12.000000,123.000000,2022.000000,23.000000


Allerdings sind die Werte in QuarCd höher als wir erwarten würden: Zürich hat nur 34 Quartiere, aber die maximale Zahl ist 123. Also sehen wir uns alle Quartiere in der Spalte an. 

Dafür benutzen wir die `.unique()`-Methode, die jeden Eintrag nur ein einziges Mal darstellt.

In [22]:
dogs["QuarLang"].unique()

array(['Altstetten', 'Leimbach', 'Oberstrass', 'Fluntern', 'Wipkingen',
       'Sihlfeld', 'Affoltern', 'Wollishofen', 'Seefeld', 'Albisrieden',
       'Witikon', 'Escher Wyss', 'Langstrasse', 'Mühlebach',
       'Gewerbeschule', 'Hottingen', 'Seebach', 'Höngg', 'Unterstrass',
       'Unbekannt (Stadt Zürich)', 'Schwamendingen-Mitte', 'Werd',
       'Friesenberg', 'Alt-Wiedikon', 'Enge', 'Hirslanden', 'Saatlen',
       'Hard', 'Hirzenbach', 'Weinegg', 'Oerlikon', 'City', 'Rathaus',
       'Lindenhof', 'Unbekannt (Kreis 4)', 'Hochschulen',
       'Unbekannt (Kreis 6)', 'Unbekannt (Kreis 1)',
       'Unbekannt (Kreis 8)'], dtype=object)

Hier sind einige Quartiere als Unbekannt aufgeführt. Aber der Name ist nicht immer gleich – sie haben in Klammern entweder den Kreis oder die Stadt stehen.

Außerdem sind mehr als die 34 Quartiere enthalten:

In [23]:
dogs["QuarLang"].unique().size

39

Auch diese Einträge wollen wir durch `pd.NA` ersetzen. Doch nicht nur in der Spalte `QuarLang` Spalte sondern auch in der Spalte `QuarCd`.

Hierfür können wir die `mask()`-Funktion gut gebrauchen, denn so können wir auf der Spalte `QuarLang` eine Maske erstellen, mit der wir alle Felder maskieren, die mit "Unbekannt" beginnen. 

In [24]:
mask = dogs["QuarLang"].str.startswith("Unbekannt")

Diese Maske wenden wir nun auf beide 

In [25]:
dogs["QuarLang"] = dogs["QuarLang"].mask(mask, pd.NA)
dogs["QuarCd"] = dogs["QuarCd"].mask(mask, pd.NA)

Nun sind nur mehr 34 Quartiere und das `pd.NA` in der Spalte:

In [28]:
dogs["QuarLang"].unique()

array(['Altstetten', 'Leimbach', 'Oberstrass', 'Fluntern', 'Wipkingen',
       'Sihlfeld', 'Affoltern', 'Wollishofen', 'Seefeld', 'Albisrieden',
       'Witikon', 'Escher Wyss', 'Langstrasse', 'Mühlebach',
       'Gewerbeschule', 'Hottingen', 'Seebach', 'Höngg', 'Unterstrass',
       <NA>, 'Schwamendingen-Mitte', 'Werd', 'Friesenberg',
       'Alt-Wiedikon', 'Enge', 'Hirslanden', 'Saatlen', 'Hard',
       'Hirzenbach', 'Weinegg', 'Oerlikon', 'City', 'Rathaus',
       'Lindenhof', 'Hochschulen'], dtype=object)

Zuletzt schreiben wir den bereinigten Datensatz auf die Festplatte. Dafür nutzen wir die `to_csv()`-Methode. 

Vergesst nicht, den Datensatz herunterzuladen – wenn das Colab-Notebook eine Zeit lang inaktiv war sind alle eure Daten gelöscht.

In [29]:
dogs.to_csv("dogs.csv")